### Full data query approach

Here we will look at getting the full data queries for the boardex and the patent data, and link everything together.

Do this tomorrow, for approach.

Create the SQL query.